In [31]:
import tkinter as tk
import time

class AscensorGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("🛗 Panel de Operario - Ascensor")
        self.piso_actual = 1
        self.estado = "reposo"
        self.puertas_abiertas = False
        self.solicitudes = []

        self.label_estado = tk.Label(root, text=f"Piso actual: {self.piso_actual} | Estado: {self.estado}", font=("Arial", 14))
        self.label_estado.pack(pady=10)

        self.label_puertas = tk.Label(root, text="🚪 Puertas cerradas", font=("Arial", 12))
        self.label_puertas.pack()

        self.frame_botones = tk.Frame(root)
        self.frame_botones.pack(pady=10)

        # Botones por piso con dirección corregida
        for piso in range(1, 6):
            if piso == 1:
                boton_subir = tk.Button(self.frame_botones, text=f"{piso} ↑", command=lambda p=piso: self.solicitar(p, "subiendo"))
                boton_subir.grid(row=piso, column=0, padx=5, pady=2)
            elif piso == 5:
                boton_bajar = tk.Button(self.frame_botones, text=f"{piso} ↓", command=lambda p=piso: self.solicitar(p, "bajando"))
                boton_bajar.grid(row=piso, column=1, padx=5, pady=2)
            else:
                boton_subir = tk.Button(self.frame_botones, text=f"{piso} ↑", command=lambda p=piso: self.solicitar(p, "subiendo"))
                boton_subir.grid(row=piso, column=0, padx=5, pady=2)
                boton_bajar = tk.Button(self.frame_botones, text=f"{piso} ↓", command=lambda p=piso: self.solicitar(p, "bajando"))
                boton_bajar.grid(row=piso, column=1, padx=5, pady=2)

        self.log = tk.Text(root, height=10, width=50)
        self.log.pack(pady=10)

        self.root.after(1000, self.mover)

    def solicitar(self, piso, direccion):
        if (piso, direccion) not in self.solicitudes:
            self.solicitudes.append((piso, direccion))
            self.log.insert(tk.END, f"📬 Solicitud en piso {piso} para {direccion}\n")
        if self.puertas_abiertas:
            self.cerrar_puertas()

    def cerrar_puertas(self):
        self.label_puertas.config(text="🚪 Puertas cerradas")
        self.puertas_abiertas = False

    def abrir_puertas(self):
        self.label_puertas.config(text=f"🚪 Puertas abiertas en piso {self.piso_actual}")
        self.puertas_abiertas = True
        self.log.insert(tk.END, f"🔔 Ding! Piso {self.piso_actual}\n")
        self.root.update()
        time.sleep(2)
        self.cerrar_puertas()

    def mover(self):
        if not self.solicitudes:
            self.estado = "reposo"
        else:
            # Buscar la solicitud más cercana
            destino, direccion = min(self.solicitudes, key=lambda s: abs(s[0] - self.piso_actual))

            if self.piso_actual < destino:
                self.piso_actual += 1
                self.estado = "subiendo"
            elif self.piso_actual > destino:
                self.piso_actual -= 1
                self.estado = "bajando"
            else:
                self.estado = "atendiendo"
                self.abrir_puertas()
                self.solicitudes.remove((destino, direccion))

            # Verificar si hay solicitud en el piso actual y en la misma dirección
            solicitudes_actuales = [s for s in self.solicitudes if s[0] == self.piso_actual and s[1] == self.estado]
            for s in solicitudes_actuales:
                self.abrir_puertas()
                self.solicitudes.remove(s)

        # Actualizar estado en la interfaz
        self.label_estado.config(text=f"Piso actual: {self.piso_actual} | Estado: {self.estado}")
        self.root.after(1000, self.mover)

# Ejecutar la interfaz
if __name__ == "__main__":
    root = tk.Tk()
    app = AscensorGUI(root)
    root.mainloop()